In [1]:
!pip install groq
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 3.5 MB/s eta 0:00:00


In [28]:
import re
import json

import groq
from groq import Groq
from google.colab import userdata
import unidecode
from tqdm import tqdm

from google.colab import userdata, drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
client = Groq(api_key=userdata.get('groq.key'))
def send_message(message):
    '''
    Dada uma mensagem, manda pelo groq para o llama3-70b-8192 e retorna a resposta do LLM.

    Input: message (str): A mensagem a ser enviada ao LLM.

    Observações: Note que para minimizar as incertezas do processo e buscando maximizar a qualidade da resposta,
    estamos usando a temperatura 0 e top_p 1.

    Output: Uma string contendo a saída do Llama3
    '''
    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": message}],
        model='llama3-70b-8192',
        temperature = 0,
        top_p = 1
    )
    return chat_completion.choices[0].message.content

In [4]:
first_query = send_message('''

Escreva as 100 possíveis pesquisas mais frequentes no Google sobre o direito no Brasil, uma por linha.

Seguem alguns exemplos já obtidos no site para referência:

1. Direito do trabalho
2. Como consultar processo
3. Porque meu nome esta no diario oficial
4. Justiça gratuita servidor publico
''')

In [5]:
base = {'buscas': re.findall(r'\d+\. (.*)',first_query)}

In [36]:
first_clean = list(map(lambda x: unidecode.unidecode(x), re.findall(r'\d+\. (.*)',first_query)))
base = {'buscas': first_clean}

##Adiciona termos relevantes do direito e pesquisas possíveis

In [10]:
second_query = send_message('''
Escreva 100 termos da área de direito, um por linha.
''')

In [37]:
term_list = re.findall(r'\d+\. (.*)', second_query)
base['buscas'] += ([unidecode.unidecode(term) for term in term_list])

In [23]:
def query_term(term):
  queries = send_message('Você é um estudante de direito estudando, você quer estudar mais sobre o termo: "{term}". Cite 3 consultas no Google que você faria, em sequência, enumeradas e uma por linha. Cada consulta deve ter ao máximo 5 palavras'.format(term=term))
  return re.findall(r'\d+\. (.*)', queries)

In [29]:
queries = []
for term in tqdm(term_list):
  queries += [unidecode.unidecode(query) for query in query_term(term)]

100%|██████████| 100/100 [03:00<00:00,  1.81s/it]


In [39]:
base['buscas'] += queries

In [41]:
with open('data/buscas.json', 'w') as jsonfile:
  json.dump(base, jsonfile)